# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
import random
import json
import requests
import pandas as pd
import numpy as np
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from train import clean_text, clean_df, preprocess_text


## Dataset

### Overview

A Kaggle dataset "60k Stack Overflow Questions with Quality Rating" has been used for this project. The dataset can be downloaded from the following link: https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate. Questions from 2016-2020 are labelled in three categories (HQ, LQ_EDIT, LQ_CLOSE), according to their quality. 45000 rows are subsetted for the project. Title and Body of each question has been appended and considered as being a single text. This text has been used as the only predictor of quality, tags are ignored. For simplicity, the task has been reduced to a binary classification between HQ and LQ, meaning that LQ_EDIT and LQ_CLOSE have been aggregated to one class, LQ.

Binary classification of the texts have been made on a embedding matrix of word counts.

In [2]:

data1 = pd.read_csv('Data/data_part1.csv')
data2 = pd.read_csv('Data/data_part2.csv')
data3 = pd.read_csv('Data/data_part3.csv')
dataset = data1.append(data2).append(data3)

dataset = clean_df(dataset)
dataset['text'] = dataset['text'].apply(clean_text)

X = preprocess_text(dataset['text'].tolist())
y = dataset['y'].to_numpy()
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.15, random_state=42)


train_dataset = pd.DataFrame(X_train)
train_dataset.columns= "X" + train_dataset.columns.astype(str)
train_dataset['y'] = y_train

val_dataset = pd.DataFrame(X_val)
val_dataset.columns= "X" + val_dataset.columns.astype(str)
val_dataset['y'] = y_val

test_dataset = pd.DataFrame(X_test)
test_dataset.columns= "X" + test_dataset.columns.astype(str)
test_dataset['y'] = y_test

if not os.path.isdir('clean_data'):
    os.mkdir('clean_data')

train_dataset.to_csv("clean_data/train_dataset.csv", index=False)
val_dataset.to_csv("clean_data/val_dataset.csv", index=False)
test_dataset.to_csv("clean_data/test_dataset.csv", index=False)



In [3]:
train_dataset.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       ...
       'X278', 'X279', 'X280', 'X281', 'X282', 'X283', 'X284', 'X285', 'X286',
       'y'],
      dtype='object', length=288)

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone-AutoML'

experiment=Experiment(ws, experiment_name)



In [5]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./clean_data', target_path='automldata', overwrite=True, show_progress=True)


Uploading an estimated of 3 files
Uploading ./clean_data/test_dataset.csv
Uploaded ./clean_data/test_dataset.csv, 1 files out of an estimated total of 3
Uploading ./clean_data/val_dataset.csv
Uploaded ./clean_data/val_dataset.csv, 2 files out of an estimated total of 3
Uploading ./clean_data/train_dataset.csv
Uploaded ./clean_data/train_dataset.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_f141e44c38344320a5cf9b2802624daf

In [6]:
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/train_dataset.csv'))
val_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/val_dataset.csv'))

test_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/test_dataset.csv'))

    

In [7]:
run = experiment.start_logging()

In [8]:
udacity_compute_name = 'comp-capstone'

compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)


try:
    compute_target = ComputeTarget(workspace=ws, name=udacity_compute_name)
    print("Found an existing compute target.")
except:
    compute_target = ComputeTarget.create(workspace=ws, 
    name=udacity_compute_name, provisioning_configuration=compute_config)
    print("Creating a new compute target.")

compute_target.wait_for_completion(show_output=True)

Creating a new compute target.
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration
AutoML configuration is done by restricting the timeout minuts to 30, choosing the task as classification, and the primary metric as AUC_weighted.
Choosing AUC as the primary metric instead of Accuracy is advantageous in minimizing True Negatives and False Positives along the way.

A single validation set has been provided as an input, not cross-validation. This has been done to increase the processing speed and use the time to try out more models and settings.


In [10]:
# TODO: Put your automl settings here
automl_settings = {
    'experiment_timeout_minutes': 50,
    'task': 'classification',
    'primary_metric': 'AUC_weighted',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    training_data=train_data,
    validation_data=val_data,
    label_column_name='y',
    compute_target=compute_target,
    **automl_settings)



In [11]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on comp-capstone with default configuration
Running on remote compute: comp-capstone


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone-AutoML,AutoML_5b92da3e-f409-46be-8143-26c1b4b5c8ef,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: ht

## Run Details
VotingEnsemble is the best algorithm, as it combines multiple algorithms like LightGBM, LogisticRegression, RandomForest, and ExtremeRandomTrees. Surprisingly LogisticRegression has a high voting power in the ensemble, meaning that the problem might be solvable also by classical statistical models.

`RunDetails` widget has been run but its output is not shown, probably due to conda enviroment related reasons. Details have been provided as pictures in Readme.md.

In [12]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run_automl, fitted_model = automl_run.get_output()



In [44]:
model_name = best_run_automl.properties['model_name']
run_algorithm = best_run_automl.properties['run_algorithm']
AUC_weighted = best_run_automl.properties['score']


'0.8680228822882288'

In [45]:
print(f"Model name:{model_name}, Algorithm:{run_algorithm}, AUC_weighted: {AUC_weighted} ")

Model name:AutoML5b92da3ef39, Algorithm:VotingEnsemble, AUC_weighted: 0.8680228822882288 


In [19]:
print(best_run_automl.properties)

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Capstone-AutoML\',\'compute_target\':\'comp-capstone\',\'subscription_id\':\'510b94ba-e453-4417-988b-fbdc37b55ca7\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_5b92da3e-f409-46be-8143-26c1b4b5c8ef_39","experiment_name":"Capstone-AutoML","workspace_name":"quick-starts-ws-144914","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group_name":"aml-quickstarts-144914"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '39', '_aml_system_scenario_identification': 'Remote.Child', '_azureml.ComputeTargetT

In [20]:
best_run_automl.get_file_names()

['accuracy_table', 'automl_driver.py', 'azureml-logs/55_azureml-execution-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/65_job_prep-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'confusion_matrix', 'logs/azureml/107_azureml.log', 'logs/azureml/azureml_automl.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/conda_env_v_1_0_0.yml', 'outputs/env_dependencies.json', 'outputs/model.pkl', 'outputs/pipeline_graph.json', 'outputs/scoring_file_v_1_0_0.py']

In [21]:
best_run_automl.download_file(best_run_automl.get_file_names()[-3])


In [22]:
# Save the downloaded file to local storage
local_path='./outputs'
os.makedirs(local_path, exist_ok=True)

best_automl_model = joblib.load(open(best_run_automl.get_file_names()[-3].split('/')[1], 'rb'))
joblib.dump(best_automl_model, 'outputs/best_automl_model.joblib')

['outputs/best_automl_model.joblib']

In [23]:
best_run_automl.get_metrics()

{'AUC_micro': 0.8682862085371924, 'accuracy': 0.7903814262023218, 'log_loss': 0.49455015453994705, 'AUC_weighted': 0.8680228822882288, 'recall_score_weighted': 0.7903814262023218, 'f1_score_macro': 0.790356311094916, 'average_precision_score_micro': 0.8659146503718596, 'recall_score_macro': 0.7903465346534653, 'AUC_macro': 0.8680228822882288, 'precision_score_weighted': 0.7904103820986601, 'average_precision_score_weighted': 0.8654422276039756, 'f1_score_micro': 0.7903814262023218, 'weighted_accuracy': 0.7904163160239592, 'balanced_accuracy': 0.7903465346534653, 'precision_score_micro': 0.7903814262023218, 'precision_score_macro': 0.7904224470554679, 'f1_score_weighted': 0.7903677270528277, 'average_precision_score_macro': 0.8654662511253289, 'norm_macro_recall': 0.5806930693069305, 'matthews_correlation': 0.5807689767476725, 'recall_score_micro': 0.7903814262023218, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_5b92da3e-f409-46be-8143-26c1b4b5c8ef_39/accuracy_table', '

In [24]:
test_dataset = pd.DataFrame(X_test)

In [25]:
test_dataset.columns = "X" + test_dataset.columns.astype(str)

In [26]:
pred_test = best_automl_model.predict(test_dataset)

In [27]:
auc_score = roc_auc_score(y_test, pred_test)
accuracy = accuracy_score(y_test, pred_test)

print('AUC Test:', auc_score)
print('Accuracy Test:', accuracy)

AUC Test: 0.7634642885086347
Accuracy Test: 0.7633333333333333


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
best_run_automl.get_file_names()

['accuracy_table', 'automl_driver.py', 'azureml-logs/55_azureml-execution-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/65_job_prep-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_f3515f24dfe8731bf990d9ca81f7d8500d43208be4a955635cb58e14c9d56daa_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'confusion_matrix', 'explanation/6601dc51/classes.interpret.json', 'explanation/6601dc51/expected_values.interpret.json', 'explanation/6601dc51/features.interpret.json', 'explanation/6601dc51/global_names/0.interpret.json', 'explanation/6601dc51/global_rank/0.interpret.json', 'explanation/6601dc51/global_values/0.interpret.json', 'explanation/6601dc51/local_importance_values_sparse.interpret.json', 'explanation/6601dc51/per_class_names/0.interpret.json', 'explanation/6601dc51/per_class_rank/0.interpret.json', 'explanation/6601d

In [29]:
print(best_run_automl.properties['model_name'])
model_name = best_run_automl.properties['model_name']

registered_model = automl_run.register_model(model_name=model_name,
description="Stackoverflow Quality Prediction", tags=None)

AutoML5b92da3ef39


In [30]:
local_path='./inference'
os.makedirs(local_path, exist_ok=True)

best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [31]:
deploy_env = Environment.get(workspace=ws, name="AzureML-AutoML")
deploy_env.save_to_directory(path="deploy_env", overwrite=True)
deploy_env = Environment.load_from_directory("deploy_env")

inference_config = InferenceConfig(entry_script="inference/score.py", environment=deploy_env)

In [32]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2, description="Stackoverflow comment quality prediction")

In [33]:
service = Model.deploy(ws, "automl-stackoverflow", [registered_model], inference_config, aciconfig)
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-16 00:03:02+00:00 Creating Container Registry if not exists..
2021-05-16 00:03:13+00:00 Registering the environment..
2021-05-16 00:03:14+00:00 Use the existing image.
2021-05-16 00:03:14+00:00 Generating deployment configuration.
2021-05-16 00:03:15+00:00 Submitting deployment to compute..
2021-05-16 00:03:22+00:00 Checking the status of deployment automl-stackoverflow..
2021-05-16 00:07:08+00:00 Checking the status of inference endpoint automl-stackoverflow.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-144914', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-144914'), name=automl-stackoverflow, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://de887f00-c979-4ffc-a4c5-21572bfe8ff5.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

In [35]:
sample_dict = {}
for i in range(287):
    
    sample_dict["X"+str(i)] = random.choice([0,1])
sample_json = json.dumps({"data":[sample_dict,]})

In [36]:
import requests
# The endpoint uri is received from the print-out of the service variable.
rest_endpoint = "http://de887f00-c979-4ffc-a4c5-21572bfe8ff5.southcentralus.azurecontainer.io/score"
headers = {"Content-Type": "application/json"}

response = requests.post(rest_endpoint, sample_json, headers=headers)

In [37]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

print(response.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [38]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "automl-stackoverflow"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

# Solution code: Enable application insights
service.update(enable_app_insights=True)

for line in logs.split('\n'):
    print(line)


2021-05-16T00:07:01,495338811+00:00 - iot-server/run 
2021-05-16T00:07:01,497598220+00:00 - rsyslog/run 
2021-05-16T00:07:01,497598220+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1509117e3dac6290ee5f15fe351d8f40/li

The Web service has been deleted:

In [39]:
service.delete()